In [ ]:
from tqdm import tqdm
import json

from videograph import VideoGraph
from utils.general import *
from utils.video_processing import *
from utils.chat_api import *
from prompts import *

from retrieve import answer_with_retrieval
from generate_memory import streaming_process_video

processing_config = json.load(open("configs/processing_config.json"))
memory_config = json.load(open("configs/memory_config.json"))

In [ ]:
# video paths can be paths to directories or paths to mp4 files
video_paths = processing_config["video_paths"]
save_dir = processing_config["save_dir"]
max_workers = 1


def process_single_video(video_path):
    video_graph = VideoGraph(**memory_config)
    streaming_process_video(video_graph, video_path)


# Process videos in parallel using ThreadPoolExecutor with max_workers limit
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Process videos in parallel using map
    list(
        tqdm(
            executor.map(process_single_video, video_paths),
            total=len(video_paths),
            desc="Processing videos",
        )
    )

In [ ]:
video_graph_path = (
    "data/mems/5-Poor-People-vs-1-Secret-Millionaire_60_5_-1_10_20_0.3_0.6.pkl"
)
video_graph = load_video_graph(video_graph_path)

question = "How might the social development of children living in military families be affected?"

answer = answer_with_retrieval(
    video_graph,
    question,
    query_num=processing_config["query_num"],
    topk=processing_config["topk"],
    mode="argmax",
)

In [ ]:
video_graph_path = (
    "data/mems/5-Poor-People-vs-1-Secret-Millionaire_60_5_-1_10_20_0.3_0.6.pkl"
)
video_graph = load_video_graph(video_graph_path)
video_graph.visualize()

# print all episodic and semantic nodes
for node_id in video_graph.text_nodes:
    print(video_graph.nodes[node_id].metadata["contents"][0])